# Verify conversion of carla dataset to COCO format

In [ ]:
import os


data_dir = "/data/shared/CARLA/multi-agent-aerial-utsa/"

frame_idx = 5  # need to index into carla (see below) to get the frame
split = "train"
idx_scene = 0
agent = 0
camera_sensor = "camera-0"

### Show carla images

In [ ]:
from avapi.carla import CarlaScenesManager


CSM = CarlaScenesManager(data_dir=os.path.join(data_dir, "raw"))
CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes[split][idx_scene])

In [ ]:
from avapi.visualize.snapshot import show_image_with_boxes


# specify viewing
frame_idx = 5  # need to index into carla (see below) to get the frame
agent = 0
camera_sensor = "camera-0"

# specify which frame
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
timestamp = CDM.get_timestamp(frame=frame, sensor=camera_sensor, agent=agent)

# get sensor data
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
objs = CDM.get_objects(frame=frame, sensor=camera_sensor, agent=agent)

# get 2D/3D boxes
boxes_3d = objs.apply_and_return("getattr", "box")
boxes_2d = boxes_3d.apply_and_return("project_to_2d_bbox", img.calibration)

# show image with 3D boxes
show_image_with_boxes(img, boxes=boxes_3d, inline=True)
show_image_with_boxes(img, boxes=boxes_2d, inline=True)

### Validate a coco conversion

In [ ]:
from my_coco import CocoDataset


annotation_path = os.path.join(data_dir, "coco", "annotations", f"{split}.json")
image_dir = os.path.join(data_dir, "coco", "images", split)

coco_dataset = CocoDataset(annotation_path, image_dir)
coco_dataset.display_categories()

In [ ]:
from IPython.display import HTML


# if you want to display the same image as above, do this
# NOTE: due to the "stride" in conversion to coco, it may not exist
# frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
# filename = CDM.get_sensor_data_filepath(frame=frame, sensor=camera_sensor, agent=agent)
# image_suffix = CDM.get_sensor_data_filepath(frame=frame, sensor=camera_sensor, agent=agent).split("/")[-1]
# image_name = f"scene-{idx_scene}-agent-{agent}-{camera_sensor}-{image_suffix}"
# image_filename = os.path.join(image_dir, image_name)
# image_id = coco_dataset.filename_to_id[image_filename]

# otherwise, just pick a random index
image_id = 25

# display the image inline
html = coco_dataset.display_image(image_id, flip_channels=True, use_url=False)
HTML(html)

In [ ]:
coco_dataset.segmentations[5]

In [ ]:
coco_dataset.segmentations[10]